In [1]:
import pandas as pd
import numpy as np

In [2]:
car_price = pd.read_csv('car price.csv')

In [3]:
car_price.shape

(11914, 16)

In [4]:
car_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

In [5]:
car_price.describe()

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP
count,11914.000000,11845.00000,11884.000000,11908.000000,11914.000000,11914.000000,11914.000000,1.191400e+04
mean,2010.384338,249.38607,5.628829,3.436093,26.637485,19.733255,1554.911197,4.059474e+04
std,7.579740,109.19187,1.780559,0.881315,8.863001,8.987798,1441.855347,6.010910e+04
min,1990.000000,55.00000,0.000000,2.000000,12.000000,7.000000,2.000000,2.000000e+03
25%,2007.000000,170.00000,4.000000,2.000000,22.000000,16.000000,549.000000,2.100000e+04
50%,2015.000000,227.00000,6.000000,4.000000,26.000000,18.000000,1385.000000,2.999500e+04
75%,2016.000000,300.00000,6.000000,4.000000,30.000000,22.000000,2009.000000,4.223125e+04
max,2017.000000,1001.00000,16.000000,4.000000,354.000000,137.000000,5657.000000,2.065902e+06


In [6]:
columns = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]

In [7]:
df = car_price[columns]
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [8]:
df.shape

(11914, 10)

In [9]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [10]:
df = df.fillna(0)

In [11]:
df.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [12]:
df.shape

(11914, 10)

In [13]:
df = df.rename(columns={'msrp': 'price'})
df['price'].head()

0    46135
1    40650
2    36350
3    29450
4    34500
Name: price, dtype: int64

# Question 1

In [14]:
df['transmission_type'].value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

In [15]:
df['transmission_type'].value_counts().index[0]

'AUTOMATIC'

# Question 2

In [16]:
df.corr()

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [17]:
corr_matr = df.corr().unstack().sort_values(ascending=False).drop_duplicates()[1:]
corr_matr

city_mpg          highway_mpg         0.886829
engine_hp         engine_cylinders    0.774851
price             engine_hp           0.650095
engine_cylinders  price               0.526274
engine_hp         year                0.338714
year              highway_mpg         0.258240
                  price               0.227590
                  city_mpg            0.198171
engine_cylinders  year               -0.040708
price             city_mpg           -0.157676
highway_mpg       price              -0.160043
engine_hp         highway_mpg        -0.415707
city_mpg          engine_hp          -0.424918
engine_cylinders  city_mpg           -0.587306
                  highway_mpg        -0.614541
dtype: float64

In [18]:
def top_entries(df):
    mat = df.corr().abs()

    # Remove duplicate and identity entries
    mat.loc[:,:] = np.tril(mat.values, k=-1)
    mat = mat[mat>0]

    # Unstack, sort ascending, and reset the index, so features are in columns
    # instead of indexes (allowing e.g. a pretty print in Jupyter).
    # Also rename these it for good measure.
    return (mat.unstack()
    .sort_values(ascending=False)
    .reset_index()
    .rename(columns={
        "level_0": "feature_a",
        "level_1": "feature_b",
        0: "correlation"
    }))

In [19]:
display(top_entries(df).iloc[0])

feature_a      highway_mpg
feature_b         city_mpg
correlation       0.886829
Name: 0, dtype: object

In [20]:
df['above_average'] = (df['price'] > df['price'].mean()) * 1

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [23]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [24]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [25]:
y_train = df_train['above_average'].values
y_val = df_val['above_average'].values
y_test = df_test['above_average'].values

In [26]:
df_train.head().T

,0,1,2,3,4
make,Mitsubishi,Kia,Lamborghini,Chevrolet,Pontiac
model,Endeavor,Borrego,Gallardo,Colorado,Vibe
year,2011,2009,2012,2016,2009
engine_hp,225.0,276.0,570.0,200.0,158.0
engine_cylinders,6.0,6.0,10.0,4.0,4.0
transmission_type,AUTOMATIC,AUTOMATIC,MANUAL,AUTOMATIC,AUTOMATIC
vehicle_style,4dr SUV,4dr SUV,Convertible,Crew Cab Pickup,4dr Hatchback
highway_mpg,19,21,20,27,26
city_mpg,15,17,12,20,20
price,33599,26245,248000,24990,20475


# Question 3

In [27]:
categorical_features = list(df.dtypes[df.dtypes == 'object'].index)

In [28]:
from sklearn.metrics import mutual_info_score

In [29]:
dict_scores = {}

for col in categorical_features:
    dict_scores[col] = round(mutual_info_score(df[col], df['above_average']), 2)

dict_scores = sorted(dict_scores.items(), key=lambda x:x[1], reverse=False)
dict_scores[0], dict_scores[0][0]

(('transmission_type', 0.02), 'transmission_type')

In [30]:
del df_train['price']
del df_val['price']
del df_test['price']

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

# Question 4

In [31]:
from sklearn.feature_extraction import DictVectorizer

In [32]:
from sklearn.linear_model import LogisticRegression

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
dv = DictVectorizer(sparse=False)

In [35]:
train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

In [36]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [37]:
y_pred = model.predict(X_val)

In [38]:
round(accuracy_score(y_pred, y_val), 2)

0.95

In [39]:
decision = (y_pred >= 0.5)
(y_val == decision).mean()

0.946286193873269

# Question 5

In [40]:
basic_accuracy = accuracy_score(y_train, model.predict(X_train))

In [41]:
basic_accuracy

0.9525741466144376

In [42]:
new_list = list(df.columns.values)
new_list.remove('above_average')
new_list.remove('price')
print(new_list)

['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg']


In [43]:
all_list = list(df.columns.values).copy()
print(all_list)

['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg', 'price', 'above_average']


In [44]:
list(df.columns.values)

['make',
 'model',
 'year',
 'engine_hp',
 'engine_cylinders',
 'transmission_type',
 'vehicle_style',
 'highway_mpg',
 'city_mpg',
 'price',
 'above_average']

In [45]:
def question5_answer(data, base_accuracy):

    answer = {}
    all_features = list(data.columns.values)
    real_features = all_features.copy()
    real_features.remove('above_average')
    real_features.remove('price')
    
    for feature in real_features:

        copy_features = all_features.copy()
        copy_features.remove('price')
        copy_features.remove(feature)

        data_full_train, data_test = train_test_split(data[copy_features], test_size=0.2, random_state=42)
        data_train, data_val = train_test_split(data_full_train, test_size=0.25, random_state=42)

        data_train = data_train.reset_index(drop=True)
        target_train = data_train['above_average'].values

        del data_train['above_average']
    
        dv = DictVectorizer(sparse=False)
    
        X_train = dv.fit_transform(data_train.to_dict(orient='records'))

        LREG = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
        LREG.fit(X_train, target_train)
        target_pred = LREG.predict(X_train)
        
        answer[feature] = base_accuracy - accuracy_score(target_train, target_pred)

    

    answer = sorted(answer.items(), key=lambda x:x[1], reverse=False)
    
    return answer, answer[0], answer[0][0]
    

In [46]:
dictionary, cell, selected_feature = question5_answer(df, basic_accuracy)

In [47]:
dictionary, cell, selected_feature

([('year', -0.003357582540570725),
  ('engine_cylinders', -0.0022383883603804833),
  ('make', 0.000699496362618901),
  ('highway_mpg', 0.0012590934527140218),
  ('city_mpg', 0.0012590934527140218),
  ('transmission_type', 0.0016787912702853625),
  ('vehicle_style', 0.004057078903189737),
  ('engine_hp', 0.009233351986569716),
  ('model', 0.03497481813094572)],
 ('year', -0.003357582540570725),
 'year')

In [48]:
selected_feature

'year'

# Question 6

In [49]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [50]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [51]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [52]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train['price'].values
y_val = df_val['price'].values
y_test = df_test['price'].values

y_train = np.log1p(y_train)
y_val = np.log1p(y_val)
y_test = np.log1p(y_test)

del df_train['price']
del df_val['price']
del df_test['price']

In [53]:
cat = list(df_train.dtypes[df_train.dtypes == 'object'].index)
num = list(df_train.dtypes[df_train.dtypes != 'object'].index)
cat, num

(['make', 'model', 'transmission_type', 'vehicle_style'],
 ['year',
  'engine_hp',
  'engine_cylinders',
  'highway_mpg',
  'city_mpg',
  'above_average'])

In [54]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [55]:
scaler = StandardScaler()
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

X_train_num = df_train[num].values
X_train_num = scaler.fit_transform(X_train_num)

X_train_cat = ohe.fit_transform(df_train[cat].values)

X_train = np.column_stack([X_train_num, X_train_cat])

X_val_num = df_val[num].values
X_val_num = scaler.transform(X_val_num)

X_val_cat = ohe.transform(df_val[cat].values)

X_val = np.column_stack([X_val_num, X_val_cat])

In [56]:
r = [0, 0.01, 0.1, 1, 10]
rmse_dict = {}

In [57]:
for element in r:
    
    model = Ridge(solver='sag', alpha=element, random_state=42, max_iter=15000)
    model.fit(X_train, y_train)
    
    X_val_num = df_val[num].values
    X_val_num = scaler.transform(X_val_num)

    X_val_cat = ohe.transform(df_val[cat].values)
    X_val = np.column_stack([X_val_num, X_val_cat])
    
    y_pred = model.predict(X_val)
    
    rmse_dict[element] = round(mean_squared_error(y_val, y_pred), 3)

In [58]:
answer = sorted(rmse_dict.items(), key=lambda x:x[1], reverse=False)
print(answer, answer[0], answer[0][0])

[(0, 0.046), (0.01, 0.046), (0.1, 0.046), (1, 0.052), (10, 0.101)] (0, 0.046) 0


In [59]:
answer[0], answer[0][0]

((0, 0.046), 0)